In [1]:
# !pip install selenium
# !pip install beautifulsoup4
# !pip install lxml

In [72]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time, random

In [131]:
driver = webdriver.Chrome('chromedriver')

In [55]:
# https://www.tokopedia.com/chochoracing/product?q=bearing

In [74]:

PAGE_NO = 1
DELAY = 3

In [127]:

def find_products(seller, page_no, div_no=4):
    time.sleep(1.5)
    first_xpath = f'//*[@id="zeus-root"]/div/div[2]/div[2]/div[{div_no}]/div'
    other_xpath = f'//*[@id="zeus-root"]/div/div[2]/div[2]/div[{div_no}]/div'
    xpath = other_xpath if page_no > 1 else first_xpath
    
    products = driver.find_element(By.XPATH, xpath)
    products_html = products.get_attribute('innerHTML')
    soup = BeautifulSoup(products_html, 'lxml')
    # print(soup.prettify())
    products = soup.find_all(attrs={"data-testid": "divProductWrapper"})
    if len(products) == 0:
        raise Exception("no products, fix xpath!")
    for product in products:
        yield {
            'seller':seller,
            'page_no': page_no,
            'name': product.find(attrs={'data-testid':'linkProductName'}).text,
            'price': product.find(attrs={'data-testid':'linkProductPrice'}).text,
            'detail_link': product.find('a').get('href')
        }

In [113]:
# list(find_products('chochoracing'))
# click next : 
def click_next(page_no,div_no):
    try: 
        
        first_xpath = f'//*[@id="zeus-root"]/div/div[2]/div[2]/div[{div_no}]/div/div[2]/div[2]/ul/a'
        other_xpath = f'//*[@id="zeus-root"]/div/div[2]/div[2]/div[{div_no}]/div/div[2]/div[2]/ul/a[2]'
        next_xpath = other_xpath if page_no > 1 else first_xpath 
        nxt=driver.find_element(By.XPATH, next_xpath)
        nxt.click() 
    except Exception as e:
        raise Exception("end of page, all done")


In [117]:
def fetch_products(seller, div_no):
    PAGE_NO = 1
    driver.get(f'https://www.tokopedia.com/{seller}/product')
    while True:
        print(f'PROCESSING PAGE : {PAGE_NO}')

        res = find_products(seller, PAGE_NO, div_no)
        products = list(res)
        df = pd.DataFrame(products)
        df.to_csv(f'data/{seller}_{PAGE_NO}.csv',index=False)
        time.sleep(random.randint(3,8))
        click_next(PAGE_NO,div_no)
        PAGE_NO += 1

In [132]:
sellers = [
    ('chochoracing',4),
    ('sakhiracing',5),
    ('btxspeedshop', 4),
    ('garage20maticsho',4),
    ('magadirracing',4),
    ('xspeedshop',5),
    ('xtrememotorshop',5),
    ('rpm58',5),
    ('adnracingproduct',5)
]
for seller in sellers:
    name, div_no = seller
    fetch_products(name, div_no)

PROCESSING PAGE : 1
PROCESSING PAGE : 2
PROCESSING PAGE : 3


Exception: end of page, all done

In [52]:
df

,seller,page_no,name,price,detail_link
0,chochoracing,1,SETANG STANG SEHER PISTON PEN PIN 15 MM NINJA ...,Rp 249.900,https://www.tokopedia.com/chochoracing/setang-...
1,chochoracing,1,BAUT MUR RING KOMSTIR SET SEGITIGA ATAS NINJA ...,Rp 164.900,https://www.tokopedia.com/chochoracing/baut-mu...
2,chochoracing,1,SEGITIGA BAWAH SHOCKBREAKER SHOCK SILVER GOLD ...,Rp 615.900,https://www.tokopedia.com/chochoracing/segitig...
3,chochoracing,1,CAP COVER TUTUP FRAME RANGKA SWING ARM AREM NI...,Rp 48.900,https://www.tokopedia.com/chochoracing/cap-cov...
4,chochoracing,1,BEARING BERING LAKER LAHER MESIN BAWAH RASIO K...,Rp 107.900,https://www.tokopedia.com/chochoracing/bearing...
...,...,...,...,...,...
75,chochoracing,1,KARET SEAL SIL ORING O RING BATANG AS ENGKOL K...,Rp 15.900,https://www.tokopedia.com/chochoracing/karet-s...
76,chochoracing,1,KARET SEAL SIL ORING O RING BATANG AS GEAR GIR...,Rp 9.900,https://www.tokopedia.com/chochoracing/karet-s...
77,chochoracing,1,KARET SEAL SIL ORING O RING GEAR GIGI RPM TAKO...,Rp 11.900,https://www.tokopedia.com/chochoracing/karet-s...
78,chochoracing,1,KARET SEAL SIL ORING O RING AS TIANG SUPER KIP...,Rp 10.900,https://www.tokopedia.com/chochoracing/karet-s...
